In [1]:
import torch
print(torch.version.cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

11.8
Using device: cuda


In [2]:
import pandas as pd
from transformers import BertTokenizer
from sklearn.preprocessing import MinMaxScaler

import torch
from torch import nn
from transformers import BertModel

# Load datasets
news_data = pd.read_csv("news.csv")

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Combine Top1-Top25 into a single string
news_data['combined_text'] = news_data.iloc[:, 2:].apply(lambda x: " ".join(x.dropna()), axis=1)

# Tokenize headlines
news_data['tokens'] = news_data['combined_text'].apply(
    lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
)

labels = news_data['Label'].values  
# Ensure this column is properly set for stock movement


In [7]:
class StockPredictor(nn.Module):
    def __init__(self):
        super(StockPredictor, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Sequential(
            nn.Linear(768, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, tokens):
        # Text embeddings
        bert_output = self.bert(**tokens).pooler_output
        return torch.sigmoid(self.fc(bert_output))


In [8]:
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, AdamW
import torch
torch.cuda.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Define Dataset Class
class NewsDataset(Dataset):
    def __init__(self, news, labels):
        self.news = news
        self.labels = torch.tensor(labels, dtype=torch.float32)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.news[idx], self.labels[idx]

# Prepare tokenized text data
train_news = news_data['tokens'].tolist()  # This contains a list of dictionaries
train_labels = news_data['Label'].values

# Create Dataset
train_dataset = NewsDataset(train_news, train_labels)

# Define Collate Function
def collate_fn(batch):
    news = {key: torch.cat([b[0][key] for b in batch], dim=0) for key in batch[0][0].keys()}  # Tokenized text
    labels = torch.stack([b[1] for b in batch])  # Labels
    return news, labels

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

# Define Model
model = StockPredictor()
loss_fn = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Training Loop
for epoch in range(5):
    model.train()
    batch_index = 0
    for batch in train_loader:
        news, labels = batch
        optimizer.zero_grad()
        outputs = model(news).squeeze()
        loss = loss_fn(outputs, labels)
        loss.backward()
        for name, param in model.named_parameters():
            if param.grad is not None:
                print(f"{name}: {param.grad.norm()}")
        optimizer.step()
        batch_index += 1
        print(f"Epoch: {epoch+1}, Batch: {batch_index}, Loss: {loss.item()}")
            

Using device: cuda
bert.embeddings.word_embeddings.weight: 0.022414805367588997
bert.embeddings.position_embeddings.weight: 0.02108394168317318
bert.embeddings.token_type_embeddings.weight: 0.04883246123790741
bert.embeddings.LayerNorm.weight: 0.004237204324454069
bert.embeddings.LayerNorm.bias: 0.003059790935367346
bert.encoder.layer.0.attention.self.query.weight: 0.004916129168123007
bert.encoder.layer.0.attention.self.query.bias: 0.00030045295716263354
bert.encoder.layer.0.attention.self.key.weight: 0.003913254477083683
bert.encoder.layer.0.attention.self.key.bias: 8.423366171239621e-11
bert.encoder.layer.0.attention.self.value.weight: 0.013453829102218151
bert.encoder.layer.0.attention.self.value.bias: 0.0025498303584754467
bert.encoder.layer.0.attention.output.dense.weight: 0.011669205501675606
bert.encoder.layer.0.attention.output.dense.bias: 0.002466656733304262
bert.encoder.layer.0.attention.output.LayerNorm.weight: 0.001222953200340271
bert.encoder.layer.0.attention.output.Lay

KeyboardInterrupt: 